DEMO - ALERT DRIVER THROUGH EMAIL IF UNHEALTHY ENGINE CONDITION FOUND.

##### To be added :
Sender email

Receiver email

Password ( App password of sender's email )

In [ ]:
import numpy as np
import pandas as pd
import shap
import joblib  # To load the model
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
import warnings

# Suppress all warnings
warnings.filterwarnings("ignore")

# Set the emergency threshold
threshold = 0.5

# Email credentials
sender_email = ""
receiver_email = ""
password = ""

# Function to send an emergency email
def send_emergency_email(critical_feature, critical_value, importance_score, health_condition_prob):
    subject = "🚨 Emergency Notification: Engine Condition Critical!"

    # Create a detailed body for the email
    body = f"""
    Dear Driver,

    🚨 Urgent: Your engine condition has been flagged as critical, and immediate attention is required.

    The predicted engine condition is Unhealthy, and the model has detected a potential failure based on the following:

    - Critical Parameter: {critical_feature}
    - Observed Value: {critical_value}
    - Feature Importance Score: {importance_score:.4f}
    - Probability of Healthy Condition: {health_condition_prob:.4f}

    The parameter listed above has the highest impact on the engine's current condition. We highly recommend pulling over and assessing the situation immediately to prevent any potential damage or further failure.

    Please seek assistance or contact support if needed.

    Stay Safe,
    The Engine Monitoring System
    """

    # Set up the MIME
    message = MIMEMultipart()
    message["From"] = sender_email
    message["To"] = receiver_email
    message["Subject"] = subject
    message.attach(MIMEText(body, "plain"))

    try:
        # Connect to the Gmail SMTP server
        with smtplib.SMTP("smtp.gmail.com", 587) as server:
            server.starttls()  # Secure the connection
            server.login(sender_email, password)  # Login to the email account
            server.sendmail(sender_email, receiver_email, message.as_string())
            print("🚨 Emergency email sent successfully!")

    except Exception as e:
        print(f"Error sending email: {e}")

# Set a random seed for reproducibility
np.random.seed(42)

# Generate random data for 20 samples
samples = {
    'Engine rpm': np.random.uniform(1000, 4000, 20),
    'Lub oil pressure': np.random.uniform(10, 80, 20),
    'Fuel pressure': np.random.uniform(30, 100, 20),
    'Coolant pressure': np.random.uniform(10, 40, 20),
    'lub oil temp': np.random.uniform(50, 120, 20),
    'Coolant temp': np.random.uniform(50, 120, 20),
}

# Create a DataFrame for the sample data
sample_data = pd.DataFrame(samples)

# Load the trained model
stacked_clf = joblib.load('engine_model.joblib')

# Predict using the loaded model
user_pred = stacked_clf.predict(sample_data)
user_pred_prob = stacked_clf.predict_proba(sample_data)[:, 1]  # probability of healthy condition

# Use SHAP to explain the model's prediction using KernelExplainer
explainer = shap.KernelExplainer(stacked_clf.predict_proba, sample_data)
shap_values = explainer.shap_values(sample_data)

# Display the prediction results and check for emergency
for i in range(len(sample_data)):
    condition = "Healthy" if user_pred[i] == 1 else "Unhealthy"
    print(f"\nPrediction Results for Sample {i+1}:")
    print(f"Predicted Engine Condition: {condition}")
    print(f"Predicted Probability of Healthy Condition: {user_pred_prob[i]:.4f}")

    # Check if an emergency notification is needed
    if user_pred_prob[i] < threshold:
        print("🚨 Emergency Notification: Engine condition is critical! Immediate attention required.")

        # Get SHAP values for the current sample
        shap_values_for_sample = shap_values[1][i]  # Assuming '1' corresponds to the unhealthy class

        # Find the most important feature
        critical_feature_index = np.argmax(np.abs(shap_values_for_sample))  # Feature with the highest absolute SHAP value
        critical_feature = sample_data.columns[critical_feature_index]
        critical_value = sample_data.iloc[i][critical_feature]
        importance_score = shap_values_for_sample[critical_feature_index]

        # Send the detailed emergency email
        send_emergency_email(critical_feature, critical_value, importance_score, user_pred_prob[i])

        # Stop processing further samples after detecting the critical condition
        break
    else:
        print("Engine is in healthy condition.")
